In [1]:
# 1분 단위로 시가 조회
    # 하나의 구간에서 거래된 첫번째 가격
# 0.2초 단위로 실시간 매수 여부 판단
# 서로 다른 두개의 로직으로 구현

In [2]:
import time
import threading
import queue
from collections import deque

import pyupbit
import datetime
from pyupbit import get_current_price, get_tickers, get_ohlcv
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [3]:
class Producer(threading.Thread):
    def __init__(self, q):
        super().__init__()
        self.q = q
    
    def run(self):
        while True:
            price = get_current_price('KRW-ADA')
            self.q.put(price)
            time.sleep(3)

In [4]:
class Consumer(threading.Thread):
    def __init__(self, q):
        super().__init__()
        self.q = q
        self.ticker = "KRW-ADA"
        
        self.ma15 = deque(maxlen=15)
        self.ma50 = deque(maxlen=50)
        self.ma120 = deque(maxlen=120)
        
        df = get_ohlcv(self.ticker, interval="minute1")
        self.ma15.extend(df['close'])
        self.ma50.extend(df['close'])
        self.ma120.extend(df['close'])
        
        print(len(self.ma15),len(self.ma50),len(self.ma120))
    
    def run(self):
        price_curr = None
        hold_flag = False # false면 매수 전
        wait_flag = False # 1분봉에 하나의 매매
        
        key0 = 'UDgx294CT64wyrhkKJ4dbk8oKnCLaze5qtrM5e0M'
        key1 = 'YHbgxjlTzF4KYpwG3Kjhi1M91YpZNSep6aPFWgHO'
        
        upbit = pyupbit.Upbit(key0, key1)
        cash = upbit.get_balance()
        print('보유현금', cash)
        
        while True:
            try:
            if not self.q.empty():
                if price_curr != None:
                    self.ma15.append(price_curr)
                    self.ma50.append(price_curr)
                    self.ma120.append(price_curr)
                    
                curr_ma15 = sum(self.ma15)/ len(self.ma15)
                curr_ma50 = sum(self.ma50)/ len(self.ma50)
                curr_ma120 = sum(self.ma120)/ len(self.ma120)
                
                price_open = self.q.get()
                price_buy = price_open * 1.01
                price_sell = price_open * 1.02
                wait_flag = False
                
            price_curr = get_current_price(self.ticker)
            
            if hold_flag == False and wait_flag == False and \
                price_curr >= price_buy and curr_ma15 >= curr_ma50 and \
                curr_ma15 <= curr_ma50 * 1.03 and curr_ma120 <= curr_ma50:
                # 0.05%
                ret = upbit.buy_marker_order(self.ticker, cash * 0.9995)
                print('매수주문', ret)
                time.sleep(1)
                
                volume = upbit.get_balance(self.ticker)
                ret = upbit.sell_limit_order(self.ticker, price_sell, volume)
                print('매도주문', ret)
                hold_flag = True
                
            if hold_flag == True:
                uncomp = upbit.get_order(self.ticker)
                if len(uncomp) == 0:
                    cash = upbit.get_balance()
                    print('매도완료', cash)
                    hold_flag = False
                    wait_flag = True
                    
            except:
                print("error")
                
            time.sleep(0.2)

In [8]:
q = queue.Queue()
# Producer(q).start()
Consumer(q).start()

15 50 120


In [8]:
key0 = 'UDgx294CT64wyrhkKJ4dbk8oKnCLaze5qtrM5e0M'
key1 = 'YHbgxjlTzF4KYpwG3Kjhi1M91YpZNSep6aPFWgHO'
upbit = pyupbit.Upbit(key0, key1)

8648.33561999

In [9]:
cash = upbit.get_balance()
cash

8648.33561999